# Plot Flowines Using Remote Greenland Ice Mapping Project Data
---

This notebook demonstrates how Greenland Ice Mapping Project can be remotely accessed to create plots along flowlines from [Felikson et al., 2020](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112), which are archived on [Zenodo](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112). The copies of the shapefiles included in this repository were downloaded in late January 2022. 

In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2
import nisardev as nisar
import os
import matplotlib.colors as mcolors
import gimpfunc as gimp
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import numpy as np
import xarray as xr
import importlib
import glob
import panel
panel.extension()

## Read Shapefiles

In the examples presented here we will use glaciers 1 through 6 in the Felikson data base, which should have downloaded as part of the notebook repository. Glacier 6 has two variants, so some extra work is needed to extract. Each glacier's flowlines are used to great `gimp.Flowlines` instances, which are saved in a dictionary, `myFlowlines` with glacier id: '0001' through 'b006'. To limit the plots to the downstream regions, the flowlines are all truncated to a length of 50km. Within each myFlowines entry (a `gimp.Flowlines` instance), the individual flowlines are maintained as a dictionary `myFlowlines['glacierId'].flowlines`.

In [ ]:
myShapeFiles = []
for i in range(1, 7):
    myShapeFiles += glob.glob(f'./shpfiles/glacier?00{i}.shp')  # Search for glaciers with ?00n where n ranges from 1 to 6
myFlowlines = {x[-8:-4]: gimp.Flowlines(shapefile=x, name=x[-8:-4], length=50e3) for x in myShapeFiles} 

The area of interest can be defined as the union of the bounds for all of the flowlines computed as shown below along with the unique set of flowline IDs across all glaciers.

In [ ]:
myBounds = {'minx': 1e9, 'miny': 1e9, 'maxx': -1e9, 'maxy': -1e9}  # Initial bounds to force reset
flowlineIDs = []  # 
for myKey in myFlowlines:
    myBounds = myFlowlines[myKey].mergeBounds(myBounds, myFlowlines[myKey].bounds)
    flowlineIDs.append(myFlowlines[myKey].flowlineIDs())
flowlineIDs = np.unique(flowlineIDs)
print(myBounds)
print(flowlineIDs)

## Search Catalog for Velocity Data

For remote access to data at NSIDC, run these cells to login with your NASA EarthData Login (see  [NSIDCLoginNotebook](https://github.com/fastice/GIMPNotebooks/blob/master/NSIDCLoginNotebook.ipynb) for further details). These cells can skipped if all data are being accessed locally. First define where the cookie files need for login are saved.

In [ ]:
env = dict(GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/.gimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/.gimp_download_cookiejar.txt'))
os.environ.update(env)

Now enter credentials. If previous valid cookie exists, login credentials can be skipped.

In [ ]:
myLogin = gimp.NASALogin()
myLogin.view()

The next cell searches the NSIDC database for the desired full Sentinel-1 based GIMP velocity mosaics. For purposes of demonstration, it comes up with a search result for annual mosaics. Use the controls to search for data ranging from 6-day to annual resolution. Note download times scale with temporal resolution. 

In [ ]:
myUrls = gimp.cmrUrls(mode='nisar')  # nisar mode excludes image and tsx products and allows only one product type at a time
myUrls.initialSearch()

## Load the Velocity Data

While the velocity data are stored as multiple files at NSIDC, they can all be combined into a single `nisarVelSeries` instance. This instance will set all the data structures up, but will not initially download the data.

In [ ]:
myVelSeries = nisar.nisarVelSeries() # Create Series
urlNames = [x.replace('vv','*').replace('.tif','') for x in myUrls.getCogs()] # Add wild card and remove trailing tiff
myVelSeries.readSeriesFromTiff(urlNames, url=True, readSpeed=False)  
myVelSeries.xr  # Add semicolon after to suppress output

For the annual data set, this step produces a ~7GB data sets, which expands to 370GB for the full data set. To avoid downloading unnessary data, the data can be subsetted using the bounding box from the flowlines. 

In [ ]:
myVelSeries.subSetVel(myBounds) # Apply subset
myVelSeries.subset # Add semicolon after to suppress output

The volume of the data set is now a far more manageable ~5MB, which is still located in the archive.  Operations can continue without downloading, but if lots of operations are going to occur, it is best to download the data upfront. 

In [ ]:
myVelSeries.loadRemote() # Load the data to memory

## Display Flowlines and Velocity

The flowlines over one of the velocity layers can be displayed as with the following block of code. In addition to plotting the flowline, a point 10 km along each flowline is plotted and saved for subsequent plots below.

In [ ]:
# set up figure and axis
fig, ax = plt.subplots(1, 1, figsize=(6,12))
glacierPoints = {}
# generate a color dict that spans all flowline ids, using method from a flowline instance
flowlineColors = list(myFlowlines.values())[0].genColorDict(flowlineIDs=flowlineIDs)
# Messy plot stuff to get glacier labels in right place
xShift = {x: 1 for x in myFlowlines} # Dict index by glacier id
xShift['0001'] = -2 # custom for jakobshavn to keep on plot
# Display the velocity map for 2020 (note picks map nearest the date given). 
# Saturate color table at 2000 to maintain slower detail (vmax=2000)
myVelSeries.displayVelForDate('2020-01-01', ax=ax, labelFontSize=10, plotFontSize=9, titleFontSize=14, 
                              vmin=0, vmax=2000, units='km') 
# Look over each glacier and plot the flowlines
for glacierId in myFlowlines:
    # Plot the flowline Match units to the map
    myFlowlines[glacierId].plotFlowlineLocations(ax=ax, units='km', colorDict=flowlineColors)
    # 
    myFlowlines[glacierId].plotGlacierName(ax=ax, units='km', xShift=xShift[glacierId], 
                                           color='w', fontsize=12,fontweight='bold', first=False)
    # Generates points 10km
    points10km = myFlowlines[glacierId].extractPoints(10, None, units='km')
    glacierPoints[glacierId] = points10km
    for key in points10km:
        ax.plot(*points10km[key], 'r.')
# Create a dict of unique labels for legend
h, l = ax.get_legend_handles_labels()
by_label = dict(zip(l, h))
# Add legend
ax.legend(by_label.values(), by_label.keys(), title='Flowline ID', ncol=2)

## Plot Central Flowlines at Different Times

This example will demonstrate plotting the nominally central flowline ('06') for each of the six years for which there are currently data. This example works for annual data. Minor modifications are needed for the legend if more frequent data are selected above. 

In [ ]:
flowlineId ='06'  # Flowline id to plot
fig, axes = plt.subplots(2, 4, figsize=(18, 9))  # Setup plot
# Loop over glaciers
for glacierId, ax in zip(myFlowlines, axes.flatten()):
    # return interpolated values as vx(time index, distance index)
    vx, vy, vv = myVelSeries.interp(*myFlowlines[glacierId].xykm(), units='km')
    # loop over each profile by time
    for speed, myDate in zip(vv, myVelSeries.time):
        ax.plot(myFlowlines[glacierId].distancekm(), speed, label=myDate.year)
    # pretty up plot
    ax.legend(ncol=2, loc='upper right')
    ax.set_xlabel('Distance (km)', fontsize=13)
    ax.set_ylabel('Speed (m/yr)', fontsize=13)
    ax.set_title(f'Glacier {glacierId}')
axes[-1, -1].axis('off'); # kill axis with no plot
plt.tight_layout()

## Plot Points Through Time

In [ ]:
This example d

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
# Loop over glaciers
for glacierId, ax in zip(glacierPoints, axes.flatten()):
    # Loop over flowlines
    for flowlineId in glacierPoints[glacierId]:
        # interpolate to get results vx(time index) for each point
        vx, vy, v = myVelSeries.interp(*glacierPoints[glacierId][flowlineId], units='km')
        ax.plot(myVelSeries.time, v, marker='o',  linestyle='-', color=flowlineColors[flowlineId],label=f'{flowlineId}')
    # pretty up plot
    ax.legend(ncol=3, loc='upper right', title='Flowline ID')
    ax.set_xlabel('year', fontsize=13)
    ax.set_ylabel('Speed (m/yr)', fontsize=13)
    ax.set_title(f'Glacier {glacierId}')
axes[-1, -1].axis('off'); # kill axis with no plot
plt.tight_layout()